# Welcome to ProgPy's Linear Model Example

The goal of this notebook is to instruct users on how to use ProgPy Model's LinearModel.

This example shows the use of the LinearModel class, a subclass of PrognosticsModel for models that can be described as a linear time series, which can be defined by the following equations:


$$
\frac{dx}{dt} = Ax + Bu + E
$$

$$
z = Cx + D
$$

$$
es = Fx + G
$$

$x$ is `state`, $u$ is `inputs`, $z$ is `output`, and $es$ is `event state`

Furthermore, Linear Models must inherit from this class and define the following properties:
* $A$: 2-D np.array[float], dimensions: n_states x n_states
* $B$: 2-D np.array[float], optional (zeros by default), dimensions: n_states x n_inputs
* $C$: 2-D np.array[float], dimensions: n_outputs x n_states
* $D$: 1-D np.array[float], optional (zeros by default), dimensions: n_outputs x 1
* $E$: 1-D np.array[float], optional (zeros by default), dimensions: n_states x 1
* $F$: 2-D np.array[float], dimensions: n_es x n_states
* $G$: 1-D np.array[float], optional (zeros by default), dimensions: n_es x 1
* __inputs__:  list[str] - `inputs` keys
* __states__:  list[str] - `state` keys
* __outputs__: list[str] - `output` keys
* __events__:  list[str] - `event` keys

We will now utilize our LinearModel to model the classical physics problem throwing an object into the air! We can create a subclass of LinearModel which will be used to simulate an object thrown, which we will call the ThrownObject Class.


First, some definitions for our Model!

    Events (2)
        | falling: The object is falling
        | impact: The object has hit the ground

    Inputs/Loading: (0)

    States: (2)
        | x: Position in space (m)
        | v: Velocity in space (m/s)

    Outputs/Measurements: (1)
        | x: Position in space (m)

Now, for our keyword arguments:

* `process_noise : Optional, float or dict[str, float]`
  * Process noise (applied at dx/next_state).
  * Can be number (e.g., .2) applied to every state, a dictionary of values for each state (e.g., {'x1': 0.2, 'x2': 0.3}), or a function (x) -> x
* `process_noise_dist : Optional, str`
  * distribution for process noise (e.g., normal, uniform, triangular)
* `measurement_noise : Optional, float or dict[str, float]`
  * Measurement noise (applied in output eqn).
  * Can be number (e.g., .2) applied to every output, a dictionary of values for each
  * output (e.g., {'z1': 0.2, 'z2': 0.3}), or a function (z) -> z
* `measurement_noise_dist : Optional, str`
  * distribution for measurement noise (e.g., normal, uniform, triangular)
* `g : Optional, float`
  * Acceleration due to gravity (m/s^2). Default is 9.81 m/s^2 (standard gravity)
* `thrower_height : Optional, float`
  * Height of the thrower (m). Default is 1.83 m
* `throwing_speed : Optional, float`
  * Speed at which the ball is thrown (m/s). Default is 40 m/s

With our definitions, we can now create the ThrownObject Model.

First, we would need to import the necessary packages.

In [ ]:
import numpy as np
from prog_models import LinearModel

: 

The goal of this notebook is to instruct users on how to use and extend the NASA PCoE Python Prognostics Model Package. 

First some background. The Prognostics Model Package is a python package for the modeling and simulation of the evolution of state for components, systems, and systems of systems, with a focus on simulating specific events. When used for prognostics, these events are typically system failures, such as a winding failure on a motor or full discharge of a battery. 

A few definitions:
* __Event__: Something that can be predicted (e.g., system failure, warning threshold). An event has either occurred or not. 
* __Event State__: Progress towards event occurring. Defined as a number where an event state of 0 indicates the event has occurred and 1 indicates no progress towards the event (i.e., fully healthy operation for a failure event). For gradually occurring events (e.g., discharge) the number will progress from 1 to 0 as the event nears. In prognostics, event state is frequently called "State of Health" or "SOH"
* __Inputs__: Control applied to the system being modeled (e.g., current drawn from a battery)
* __Outputs__: Measured sensor values from a system (e.g., voltage and temperature of a battery), outputs can be estimated from system state
* __States__: Internal parameters (typically hidden states) used to represent the state of the system- can be the same as inputs/outputs but do not have to be. 

The `prog_models` package has the following structure
* `prog_models.data_models` - package containing algorithms for data-driven models, and parent class `prog_models.data_models.DataModel`
* `prog_models.datasets` - package containing tools for downloading a few relevant datasets
* `prog_models.models.*` - implemented models (e.g., pump, valve, battery)
* `prog_models.LinearModel` - Parent class for simple linear models
* `prog_models.PrognosticsModel` - parent class for all prognostics models - defines interfaces that a model must implement, and tools for simulating said model 

In addition to the `prog_models` package, this repo includes many examples illustrating how to use the package (see `examples/`), a template for implementing a new model (`prog_model_template`), documentation (<https://nasa.github.io/progpy>), and this tutorial (`tutorial.ipynb`).

Before you start, make sure to install prog_models using the following command:

    pip install prog_models

Now let's get started with some examples